### Install Required Packages:

In [ ]:
# !pip install -q trl

### Import Libraries:

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, pipeline, logging
from trl import SFTTrainer
from peft import LoraConfig, PeftModel, TaskType
import torch
import json
from datasets import Dataset

import warnings
warnings.filterwarnings('ignore')
logging.set_verbosity(logging.CRITICAL)

### Load the Dataset:

In [ ]:
# Load dataset
with open("/content/custom_quiz_dataset.jsonl", "r") as f:
    data = [json.loads(line) for line in f if line.strip()]

# Convert to decoder-only format: input + output combined as one sequence
for d in data:
    output = d["output"]
    d["text"] = f"{d['input']}\nQuestion: {output['Question']}\nAnswer: {output['Answer']}"

dataset = Dataset.from_list(data)

In [ ]:
dataset

### Load Model and Tokenizer:

In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# base_model
base_model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)

In [ ]:
base_model

### Baseline Generation:

In [ ]:
pipe = pipeline("text-generation", model=base_model, tokenizer=tokenizer, max_length=80)

# Define the prompt
ques_type = "multiple choice question"
topic = "Physics"
prompt = f"Generate a {ques_type} on {topic}"

# Generate and print result
result = pipe(prompt)
print(result[0]['generated_text'])

### Configure LoRA:

In [ ]:
lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # Adjust based on model architecture
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

### Data Preprocessing:

In [ ]:
def preprocess(sample):
  return sample['text']

###  Training Configuration:

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "True"

In [ ]:
# Training Arguments
training_args = TrainingArguments(
    output_dir="./tinyllama-quiz",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    logging_steps=50,
    save_strategy="epoch",
    report_to="none",
    fp16=True,
)

### Initialize Trainer:

In [ ]:
# Trainer
trainer = SFTTrainer(
    model = base_model,
    train_dataset = dataset,
    peft_config = lora_config,
    formatting_func = preprocess,
    args = training_args,
)

### Start Training:

In [ ]:
trainer.train()

### Save the Fine-Tuned Model

In [ ]:
new_model_name = "tinyllama_finetuned_lora"

trainer.model.save_pretrained(new_model_name)
tokenizer.save_pretrained(new_model_name)

In [ ]:
pipe = pipeline(task="text-generation", model=new_model_name, tokenizer=new_model_name, max_length=80)

### Inference on the test prompt

In [ ]:
ques_type = "fill in the blanks"
topic = "Science"

prompt = f"Generate a {ques_type} on {topic}"
result = pipe(prompt)

# print(result)
print(result[0]['generated_text'])

In [ ]:
prompt = "Generate a multiple choice question on Physics"
result = pipe(prompt)
print(result[0]['generated_text'])